## 图像卷积
互相关运算

In [4]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[0]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

## 验证上述二维互相关运算的输出

In [5]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 实现二维卷积层

In [6]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return corr2d(X, self.weight ) + self.bias
    

## 卷积层的简单应用：检测图像中不同颜色的边缘

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [11]:
K = torch.tensor([[1.0, -1.0]])

## 输出```y```中的1代表白色到黑色的边缘，-1代表从黑色到白色的边缘

In [17]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

## 卷积核```K```只能检测垂直边缘

In [19]:
# corr2d(X.t(), K)

## 学习由```X```生成```Y```的卷积核


In [25]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(40):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'Batch {i+1}, loss {l.sum():.3f}')
        

Batch 2, loss 4.094
Batch 4, loss 1.083
Batch 6, loss 0.344
Batch 8, loss 0.124
Batch 10, loss 0.048
Batch 12, loss 0.019
Batch 14, loss 0.008
Batch 16, loss 0.003
Batch 18, loss 0.001
Batch 20, loss 0.001
Batch 22, loss 0.000
Batch 24, loss 0.000
Batch 26, loss 0.000
Batch 28, loss 0.000
Batch 30, loss 0.000
Batch 32, loss 0.000
Batch 34, loss 0.000
Batch 36, loss 0.000
Batch 38, loss 0.000
Batch 40, loss 0.000


## 所学的卷积核的权重张量

In [26]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0000, -1.0000]])